In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy, scipy as sklearn, librosa, urllib
import librosa.display
from IPython.display import Audio
import json 
import seaborn as sns
from sklearn.cluster import KMeans
import csv
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, recall_score, precision_score, f1_score
import keras
from sklearn.decomposition import PCA

from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler
from os import path

from itertools import cycle
from sklearn.manifold import TSNE
from sklearn.metrics import roc_curve, auc, silhouette_score,roc_auc_score, precision_recall_fscore_support
from tqdm import tqdm
import opensmile


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input

from tensorflow.keras import regularizers
from joblib import dump, load
from sklearn.metrics import mean_squared_error
from tensorflow.keras.datasets import mnist

from keras.models import load_model

import random



In [2]:
def get_emo_df(emotion, dataframe):
    emo = []
    tot = 0
    print(f"Processing {emotion} ...")
    for id, name in zip(dataframe["filename"],dataframe["label"]):
        if f"{emotion}" in name:
            emo.append(id)
            tot += 1
    print(f"Processed {tot} file for {emotion}")


    mask = dataframe['filename'].isin(emo)
    emo_df = dataframe[mask].reset_index()
    emo_df = emo_df.drop(['index'], axis=1)
    return emo_df


In [3]:
def get_data(df,seed):
    X = pd.concat(df["features"].tolist(), ignore_index=True)
    _X= (X-X.min())/(X.max()-X.min())
    


    X_train, X_test = train_test_split(_X, test_size=.2, random_state=seed)
    X_train, X_val = train_test_split(X_train, test_size=.1, random_state=seed)
    # print(len(X_train), len(X_val), len(X_test))
    return X_train, X_val, X_test

In [4]:
# clear_df = pd.read_pickle("../utils/data/clear_functional.pkl")
# seed = 83
clear_df = pd.read_pickle("../utils/data/clear_functional_emovo.pkl")
seed = 10

# clear_df = pd.read_pickle("../utils/data/pitch_functional_demos.pkl")
# clear_df = pd.read_pickle("../utils/data/pitch_functional_emovo.pkl")
labels = clear_df["label"].unique()
labels

array(['disgust', 'joy', 'neutrality', 'fear', 'anger', 'surprise',
       'sadness'], dtype=object)

In [5]:
df = []

for i,emotion in enumerate(labels):
    df.append(get_emo_df(emotion,clear_df))

Processing disgust ...
Processed 84 file for disgust
Processing joy ...
Processed 84 file for joy
Processing neutrality ...
Processed 84 file for neutrality
Processing fear ...
Processed 84 file for fear
Processing anger ...
Processed 84 file for anger
Processing surprise ...
Processed 84 file for surprise
Processing sadness ...
Processed 83 file for sadness


In [6]:
train = []
val = []
test = []
print(seed)
for data in df:
    tr, v, te = get_data(data, seed)
    train.append(tr)
    val.append(v)
    test.append(te)

10


In [7]:
def get_rec_error(ae, seed, flag):
    errors = {}
    for i in labels:
        errors[i] = None
    for i in range(len(labels)):
        data = df[i]
        _X_train, _X_val, _X_test = get_data(data, seed)
        if flag:
            _X_test = _X_train
        reconstructed_X = ae.predict(_X_test)
        errors[labels[i]] = mean_squared_error(_X_test,reconstructed_X)
        # mse_per_feature = ((_X_test - reconstructed_X)**2).mean(axis=0)
        # errors[labels[i]] = np.mean(mse_per_feature)
    return errors
        

In [8]:
def obtain_cm(y_true, y_pred, labels, title):
    cm = confusion_matrix(y_true,y_pred)

    g = sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=True)
    g.xaxis.set_ticklabels(labels, rotation = 15,fontsize=12)
    g.yaxis.set_ticklabels(labels, rotation = 30,fontsize=12)
    g.set_xlabel("Predicted Label", fontsize=14)
    g.set_ylabel("True Label", fontsize = 14)
    g.set_title(title)
    
    name = title + ".png"
    plt.tight_layout()
    plt.show()
    # g.figure.savefig(name)

# g.set_title(data, fontsize = 18)

In [18]:
from sklearn.metrics import mean_squared_error

def obtain_predictions(test, autoencoders):
    y_true = []

    y_pred = []
    errors = []
    _test = []
    for n in range(len(test)):
        _test.append(test[n].to_numpy())
    for n in tqdm(range(len(df))):
        X_test = _test[n]
        cls = []

        for i in (range(len(X_test))):
            _er = []
            for encoder in autoencoders:
            #print(labels[x])
                _er.append(mean_squared_error(X_test[i],encoder.predict(X_test[i].reshape(-1,88), verbose=0).reshape(88,)))
            errors.append(_er)
            y_pred.append(np.argmin(_er))
            y_true.append(n)

        print(np.array(errors).shape)
    return errors, y_true, y_pred

    #pred.append(cls)
    # print(cls)

In [10]:
def load_models(model_path,labels, i ):
    model = (model_path + labels[i] + ".keras")
    print(model)
    return load_model(model)

In [11]:
EMOVO_model_path = "../utils/paper/models/EMOVO/ae_"
DEMoS_model_path = "../utils/paper/models/DEMoS/ae_"

autoencoders = []
for i,emotion in enumerate(labels):
    autoencoders.append(load_models(EMOVO_model_path, labels, i))

../utils/paper/models/EMOVO/ae_disgust.keras
../utils/paper/models/EMOVO/ae_joy.keras
../utils/paper/models/EMOVO/ae_neutrality.keras
../utils/paper/models/EMOVO/ae_fear.keras
../utils/paper/models/EMOVO/ae_anger.keras
../utils/paper/models/EMOVO/ae_surprise.keras
../utils/paper/models/EMOVO/ae_sadness.keras


In [ ]:
def obtain_predictions(test, autoencoders):
    y_true = []

    y_pred = []
    errors = []
    _test = []

    for n in range(len(test)):
        _test.append(test[n].to_numpy())
    for n in tqdm(range(len(df))):
        X_test = _test[n]
    
        for i in (range(len(X_test))):
            _er = []
            for encoder in autoencoders:
            #print(labels[x])
                _er.append(mean_squared_error(X_test[i],encoder.predict(X_test[i].reshape(-1,88), verbose=0).reshape(88,)))
            errors.append(_er)
            y_pred.append(np.argmin(_er))
            y_true.append(n)

        print(np.array(errors).shape)
    return errors, y_true, y_pred

    #pred.append(cls)
    # print(cls)

In [19]:
test_errors, y_true, y_pred = obtain_predictions(test, autoencoders)
# obtain_cm(y_true, y_pred, labels, "EMOVO on Test_")

 14%|█▍        | 1/7 [00:04<00:27,  4.62s/it]

(17, 7)


 29%|██▊       | 2/7 [00:09<00:23,  4.72s/it]

(34, 7)


 43%|████▎     | 3/7 [00:14<00:19,  4.77s/it]

...

(85, 7)


 86%|████████▌ | 6/7 [00:29<00:04,  4.96s/it]

(102, 7)


100%|██████████| 7/7 [00:34<00:00,  4.94s/it]

(119, 7)


In [23]:
train_errors = np.array(test_errors)

In [25]:
t_e = train_errors.reshape(-1,17,7)

In [31]:
test_errors[0:17]

[[0.020770293,
  0.057478856,
  0.03713328,
  0.054582085,
  0.046577394,
  0.038084034,
  0.04938738],
 [0.013267235,
  0.040953383,
  0.036191657,
  0.025775492,
  0.041210607,
  0.01950435,
  0.039127827],
 [0.010056591,
  0.04798721,
  0.03235099,
  0.031125372,
  0.036101207,
  0.024937494,
  0.026530234],
 [0.0062946454,
  0.052785482,
  0.026283043,
  0.038767874,
  0.03830336,
  0.021587001,
  0.02841103],
 [0.01714139,
  0.058725074,
  0.035159796,
  0.03838003,
  0.04608853,
  0.026665818,
  0.038464334],
 [0.00863939,
  0.048564173,
  0.029012673,
  0.02998183,
  0.037381265,
  0.0206675,
  0.022457844],
 [0.016770052,
  0.06308653,
  0.032512583,
  0.044479832,
  0.04843135,
  0.03956221,
  0.03483697],
 [0.008803836,
  0.04362712,
  0.026480513,
  0.028956449,
  0.0384306,
  0.026399232,
  0.04035507],
 [0.021150185,
  0.06775555,
  0.046948694,
  0.058160223,
  0.05096144,
  0.04123442,
  0.043400727],
 [0.026396139,
  0.043403905,
  0.03698847,
  0.044567022,
  0.0452915

In [33]:
t_e[0]

array([[0.02077029, 0.05747886, 0.03713328, 0.05458209, 0.04657739,
        0.03808403, 0.04938738],
       [0.01326723, 0.04095338, 0.03619166, 0.02577549, 0.04121061,
        0.01950435, 0.03912783],
       [0.01005659, 0.04798721, 0.03235099, 0.03112537, 0.03610121,
        0.02493749, 0.02653023],
       [0.00629465, 0.05278548, 0.02628304, 0.03876787, 0.03830336,
        0.021587  , 0.02841103],
       [0.01714139, 0.05872507, 0.0351598 , 0.03838003, 0.04608853,
        0.02666582, 0.03846433],
       [0.00863939, 0.04856417, 0.02901267, 0.02998183, 0.03738127,
        0.0206675 , 0.02245784],
       [0.01677005, 0.06308653, 0.03251258, 0.04447983, 0.04843135,
        0.03956221, 0.03483697],
       [0.00880384, 0.04362712, 0.02648051, 0.02895645, 0.0384306 ,
        0.02639923, 0.04035507],
       [0.02115018, 0.06775555, 0.04694869, 0.05816022, 0.05096144,
        0.04123442, 0.04340073],
       [0.02639614, 0.0434039 , 0.03698847, 0.04456702, 0.04529152,
        0.03308646, 0.0

In [37]:
t_e[0][0]

array([0.02077029, 0.05747886, 0.03713328, 0.05458209, 0.04657739,
       0.03808403, 0.04938738], dtype=float32)

In [ ]:
next = 
for errors in (t_e[0]):
    print(errors)

[0.02077029 0.05747886 0.03713328 0.05458209 0.04657739 0.03808403
 0.04938738]
[0.01326723 0.04095338 0.03619166 0.02577549 0.04121061 0.01950435
 0.03912783]
[0.01005659 0.04798721 0.03235099 0.03112537 0.03610121 0.02493749
 0.02653023]
[0.00629465 0.05278548 0.02628304 0.03876787 0.03830336 0.021587
 0.02841103]
[0.01714139 0.05872507 0.0351598  0.03838003 0.04608853 0.02666582
 0.03846433]
[0.00863939 0.04856417 0.02901267 0.02998183 0.03738127 0.0206675
 0.02245784]
[0.01677005 0.06308653 0.03251258 0.04447983 0.04843135 0.03956221
 0.03483697]
[0.00880384 0.04362712 0.02648051 0.02895645 0.0384306  0.02639923
 0.04035507]
[0.02115018 0.06775555 0.04694869 0.05816022 0.05096144 0.04123442
 0.04340073]
[0.02639614 0.0434039  0.03698847 0.04456702 0.04529152 0.03308646
 0.04723521]
[0.01542581 0.05929357 0.03730005 0.0349131  0.04169695 0.02323387
 0.03761248]
[0.02765572 0.06835905 0.03792909 0.04964529 0.04467317 0.03374411
 0.05745163]
[0.01111976 0.05143696 0.04209951 0.0370233